In [ ]:
!pip install diffusers torch torchvision pipeline transformers accelerate safetensors sentencepiece spaces peft

In [ ]:
Versao do numpy:

In [ ]:
!pip install numpy==1.25.2

Caso tenha problema com o gradio:

In [ ]:
!pip install --upgrade --force-reinstall gradio

Token para Acesso ao stabilityai/stable-diffusion-3.5-large-turbo

In [ ]:
pip install huggingface_hub


In [ ]:
from huggingface_hub import login

login(token="")


In [2]:
import os
import random
import uuid
import gradio as gr
import numpy as np
from PIL import Image
import spaces
import torch
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
from typing import Tuple

css = '''
.gradio-container{max-width: 575px !important}
h1{text-align:center}
footer {
    visibility: hidden
}
'''

DESCRIPTIONXX = """## Geração de Imagem com IA Disciplina de PI4"""

examples = [
    "A futuristic cityscape with neon lights and flying cars, cyberpunk theme, 4k, ultra-detailed --v 6.0 --style raw5",
    "A majestic lion standing on a rocky cliff during sunset, hyper-realistic, 4K resolution --v 6.0 --style raw5",
    "A cozy winter cabin with a fireplace glowing warmly, snow falling outside, peaceful atmosphere --ar 16:9 --v 6.0 --style raw5",
    "A surreal dreamscape with floating islands and vibrant flowers, ethereal theme, vibrant colors --v 6.0 --style raw5",
]

MODEL_OPTIONS = {
    "Stable Diffusion 1.5": "runwayml/stable-diffusion-v1-5",
    "Stable Diffusion 2.1": "stabilityai/stable-diffusion-2-1",
    # Outros modelos devem ser habilitados manualmente

    # "Stable Diffusion 3.5-large-turbo": "stabilityai/stable-diffusion-3-5-large-turbo",
    # "LIGHTNING V5.0": "SG161222/RealVisXL_V5.0_Lightning",
    # "LIGHTNING V4.0": "SG161222/RealVisXL_V4.0_Lightning",
}

MAX_IMAGE_SIZE = int(os.getenv("MAX_IMAGE_SIZE", "4096"))
USE_TORCH_COMPILE = os.getenv("USE_TORCH_COMPILE", "0") == "1"
ENABLE_CPU_OFFLOAD = os.getenv("ENABLE_CPU_OFFLOAD", "0") == "1"
BATCH_SIZE = int(os.getenv("BATCH_SIZE", "1"))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

style_list = [
    {
        "name": "3840 x 2160",
        "prompt": "hyper-realistic 8K image of {prompt}. ultra-detailed, lifelike, high-resolution, sharp, vibrant colors, photorealistic",
        "negative_prompt": "cartoonish, low resolution, blurry, simplistic, abstract, deformed, ugly",
    },
    {
        "name": "2560 x 1440",
        "prompt": "hyper-realistic 4K image of {prompt}. ultra-detailed, lifelike, high-resolution, sharp, vibrant colors, photorealistic",
        "negative_prompt": "cartoonish, low resolution, blurry, simplistic, abstract, deformed, ugly",
    },
    {
        "name": "HD+",
        "prompt": "hyper-realistic 2K image of {prompt}. ultra-detailed, lifelike, high-resolution, sharp, vibrant colors, photorealistic",
        "negative_prompt": "cartoonish, low resolution, blurry, simplistic, abstract, deformed, ugly",
    },
    {
        "name": "Style Zero",
        "prompt": "{prompt}",
        "negative_prompt": "",
    },
]

styles = {k["name"]: (k["prompt"], k["negative_prompt"]) for k in style_list}
DEFAULT_STYLE_NAME = "3840 x 2160"
STYLE_NAMES = list(styles.keys())

def apply_style(style_name: str, positive: str, negative: str = "") -> Tuple[str, str]:
    if style_name in styles:
        p, n = styles.get(style_name, styles[DEFAULT_STYLE_NAME])
    else:
        p, n = styles[DEFAULT_STYLE_NAME]

    if not negative:
        negative = ""
    return p.replace("{prompt}", positive), n + negative

def load_and_prepare_model(model_id):
    if model_id == "stabilityai/stable-diffusion-3.5-large-turbo":
        pipe = DiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        ).to(device)
    else:
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            use_safetensors=True,
            add_watermarker=False,
        ).to(device)
        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

    if USE_TORCH_COMPILE:
        pipe.compile()

    if ENABLE_CPU_OFFLOAD:
        pipe.enable_model_cpu_offload()

    return pipe

# Preload and compile both models
models = {key: load_and_prepare_model(value) for key, value in MODEL_OPTIONS.items()}

MAX_SEED = np.iinfo(np.int32).max

def save_image(img):
    unique_name = str(uuid.uuid4()) + ".png"
    img.save(unique_name)
    return unique_name

def randomize_seed_fn(seed: int, randomize_seed: bool) -> int:
    if randomize_seed:
        seed = random.randint(0, MAX_SEED)
    return seed

@spaces.GPU(duration=60, enable_queue=True)
def generate(
    model_choice: str,
    prompt: str,
    negative_prompt: str = "",
    use_negative_prompt: bool = False,
    style_selection: str = DEFAULT_STYLE_NAME,
    seed: int = 1,
    width: int = 1024,
    height: int = 1024,
    guidance_scale: float = 3,
    num_inference_steps: int = 25,
    randomize_seed: bool = False,
    use_resolution_binning: bool = True,
    num_images: int = 1,
    progress=gr.Progress(track_tqdm=True),
):
    global models
    pipe = models[model_choice]

    seed = int(randomize_seed_fn(seed, randomize_seed))
    generator = torch.Generator(device=device).manual_seed(seed)

    prompt, negative_prompt = apply_style(style_selection, prompt, negative_prompt)

    options = {
        "prompt": [prompt] * num_images,
        "negative_prompt": [negative_prompt] * num_images if use_negative_prompt else None,
        "width": width,
        "height": height,
        "guidance_scale": guidance_scale,
        "num_inference_steps": num_inference_steps,
        "generator": generator,
        "output_type": "pil",
    }

    if use_resolution_binning:
        options["use_resolution_binning"] = True

    images = []
    for i in range(0, num_images, BATCH_SIZE):
        batch_options = options.copy()
        batch_options["prompt"] = options["prompt"][i:i+BATCH_SIZE]
        if "negative_prompt" in batch_options:
            batch_options["negative_prompt"] = options["negative_prompt"][i:i+BATCH_SIZE]
        images.extend(pipe(**batch_options).images)

    image_paths = [save_image(img) for img in images]
    return image_paths, seed

# Frontend
with gr.Blocks(css=css, theme="bethecloud/storj_theme") as demo:
    gr.Markdown(DESCRIPTIONXX)
    with gr.Row():
        prompt = gr.Text(
            label="Prompt",
            show_label=False,
            max_lines=1,
            placeholder="Adicione um texto para gerar a imagem",
            container=False,
        )
        run_button = gr.Button("Gerar", scale=0)
    result = gr.Gallery(label="Result", columns=1, show_label=False)

    with gr.Row():
        model_choice = gr.Dropdown(
            label="Seleção de modelo⬇️",
            choices=list(MODEL_OPTIONS.keys()),
            value="Stable Diffusion 2.1"
        )

    with gr.Accordion("Explicação dos parametros", open=False, visible=True):
        gr.Markdown("""
        ### Explicação do Modelo e Parâmetros

        - **Modelo**: O modelo Stable Diffusion é uma rede neural que gera imagens a partir de descrições textuais. O modelo é baseado em difusão e usa uma técnica chamada "guidance" para gerar imagens mais relevantes para o prompt fornecido.

        - **Prompt**: O texto fornecido descreve o que você deseja na imagem. Quanto mais detalhado o prompt, mais específica será a imagem gerada.

        - **Prompt Negativo**: Descreve o que você **não** quer na imagem. Por exemplo, "sem ruídos", "sem deformações".

        - **Seed**: Um número que determina a aleatoriedade da geração. Usar a mesma seed com o mesmo prompt gera a mesma imagem.

        - **Estilo**: Ajusta o estilo da imagem gerada (resolução, detalhamento, etc.).

        - **Escalamento de orientação**: Controla a intensidade com que o modelo segue o prompt. Um valor maior resulta em uma imagem mais fiel ao texto, enquanto um valor menor pode ser mais criativo.

        - **Passos de Inferência**: Determina quantos passos o modelo tomará para gerar a imagem. Mais passos podem resultar em uma imagem de melhor qualidade, mas são mais lentos.
        """)

    with gr.Accordion("Opções Avançadas", open=False, visible=True):
        style_selection = gr.Radio(
            show_label=True,
            container=True,
            interactive=True,
            choices=STYLE_NAMES,
            value=DEFAULT_STYLE_NAME,
            label="Estilo",
        )
        num_images = gr.Slider(
            label="Número de Imagens",
            minimum=1,
            maximum=5,
            step=1,
            value=1,
        )
        with gr.Row():
           with gr.Column(scale=1):
                use_negative_prompt = gr.Checkbox(label="Use negative prompt", value=True)
                negative_prompt = gr.Text(
                    label="Negative prompt",
                    max_lines=5,
                    lines=4,
                    placeholder="Enter a negative prompt",
                    value="(deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4), disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation",
                    visible=True,
                )
        with gr.Row():
            with gr.Column(scale=1):
                randomize_seed = gr.Checkbox(value=False, label="Seed aleatória?")
        with gr.Row():
            with gr.Column(scale=1):
                seed = gr.Slider(label="Seed", value=1, minimum=0, maximum=MAX_SEED)
            with gr.Column(scale=1):
                guidance_scale = gr.Slider(
                    label="Escalamento de Orientação",
                    minimum=0,
                    maximum=30,
                    step=0.1,
                    value=7.5,
                )
        with gr.Row():
            with gr.Column(scale=1):
                num_inference_steps = gr.Slider(
                    label="Passos de Inferência",
                    minimum=1,
                    maximum=100,
                    step=1,
                    value=25,
                )
            with gr.Column(scale=1):
                width = gr.Slider(
                    label="Largura (px)",
                    minimum=512,
                    maximum=MAX_IMAGE_SIZE,
                    value=1024,
                    step=128,
                )
                height = gr.Slider(
                    label="Altura (px)",
                    minimum=512,
                    maximum=MAX_IMAGE_SIZE,
                    value=1024,
                    step=128,
                )
    with gr.Accordion("Explicação dos Modelos", open=False, visible=True):
        gr.Markdown("""
    ### Explicação dos Modelos de Stable Diffusion e Lightning

    - **Stable Diffusion 1.5**: Este modelo é uma versão anterior e ainda muito popular do Stable Diffusion. Ele é projetado para gerar imagens com boa qualidade e velocidade. A ênfase está em um equilíbrio entre a fidelidade ao prompt e a criatividade. Ideal para gerar imagens com um estilo mais clássico e bem definido.

    - **Stable Diffusion 2.1**: Esta versão apresenta melhorias significativas em relação à 1.5, incluindo uma maior capacidade de gerar imagens mais nítidas e detalhadas, com um melhor controle sobre a composição e os elementos da imagem. O modelo 2.1 é mais refinado em termos de geração de paisagens e cenas realistas, além de ser mais robusto em relação a prompts mais complexos.

    - **Stable Diffusion 3.5-large-turbo (opcional)**: A versão 3.5, embora não esteja habilitada no código, é uma versão mais recente que traz aprimoramentos no entendimento semântico dos prompts, resultando em imagens ainda mais realistas e alinhadas com descrições mais abstratas. Este modelo é adequado para casos onde a precisão na interpretação do prompt é crucial. A principal vantagem é a geração de imagens com maior fidelidade a detalhes complexos e relações contextuais.

    - **Lightning V4.0 (opcional)**: Este modelo, com aproximadamente 1,5 bilhões a 2 bilhões de parâmetros, é uma versão avançada da arquitetura de difusão, capaz de gerar imagens de alta qualidade com maior precisão no alinhamento dos elementos do prompt. Ele é otimizado para fornecer mais controle sobre os detalhes das imagens e é adequado para situações onde se deseja mais realismo e precisão nas imagens geradas.

    - **Lightning V5.0 (opcional)**: Com 2 bilhões a 3 bilhões de parâmetros, o Lightning V5.0 oferece uma qualidade excepcional na geração de imagens, sendo capaz de produzir imagens altamente realistas e detalhadas. O modelo é ideal para geração de imagens com maior complexidade e fidelidade ao prompt. A sua arquitetura avançada permite uma compreensão mais refinada de descrições complexas e uma geração de imagens muito mais fiel e precisa, com resultados de alta resolução e detalhes.

    ### Como os Modelos Afetam os Resultados:

    - **Fidelidade ao Prompt**: Modelos mais novos, como o 2.1, 3.5, Lightning V4.0 e V5.0, tendem a gerar imagens mais detalhadas e alinhadas com a descrição textual fornecida, resultando em maior fidelidade ao prompt. Modelos como o **Lightning V5.0** apresentam uma excelente capacidade de gerar imagens altamente precisas, refletindo bem as nuances do texto fornecido.

    - **Criatividade vs. Precisão**: Modelos mais antigos, como o 1.5, podem ser mais criativos e gerar imagens mais artísticas e imprevisíveis, enquanto os modelos mais recentes e maiores (como o **Lightning V5.0**) tendem a ser mais precisos, com um maior foco na criação de imagens realistas e fiéis ao prompt.

    - **Desempenho e Velocidade**: O modelo **Stable Diffusion 1.5** pode ser mais rápido em termos de desempenho, especialmente para imagens simples, enquanto modelos mais avançados como o **Lightning V5.0** ou **Stable Diffusion 2.1** podem gerar imagens mais refinadas, porém com um custo computacional maior, resultando em um tempo de processamento mais longo e maior demanda por recursos (memória e poder de GPU).

    ### Tamanho dos Modelos e Parâmetros:

    - **Stable Diffusion 1.5**: Aproximadamente **890 milhões** de parâmetros. Ideal para imagens com estilo clássico e boa velocidade de geração.
    - **Stable Diffusion 2.1**: Aproximadamente **1,2 bilhões** de parâmetros. Oferece maior detalhamento, controle e refinamento em relação ao 1.5.
    - **Stable Diffusion 3.5**: Aproximadamente **1,28 bilhões** de parâmetros. Melhor para geração de imagens com precisão em detalhes complexos e abstrações.
    - **Lightning V4.0**: Aproximadamente **1,5 bilhões a 2 bilhões** de parâmetros. Melhor controle sobre a composição e maior realismo nas imagens geradas.
    - **Lightning V5.0**: Aproximadamente **2 bilhões a 3 bilhões** de parâmetros. Excelente para imagens com alta fidelidade e resolução, oferecendo precisão e realismo excepcionais.

    O número de parâmetros de cada modelo afeta a **qualidade da imagem gerada**, a **fidelidade ao prompt**, e o **tempo de processamento**. Modelos maiores, como os **Lightning V5.0** e **Stable Diffusion 3.5**, tendem a gerar imagens mais realistas e detalhadas, mas exigem mais poder computacional para processar as imagens.

    """)

    # Generate
    run_button.click(
        fn=generate,
        inputs=[
            model_choice,
            prompt,
            negative_prompt,
            use_negative_prompt,
            style_selection,
            seed,
            width,
            height,
            guidance_scale,
            num_inference_steps,
            randomize_seed,
            num_images,
        ],
        outputs=[result, seed],
    )


demo.queue().launch()


Keyword arguments {'add_watermarker': False} are not expected by StableDiffusionPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 7/7 [00:20<00:00,  2.92s/steps]
Keyword arguments {'add_watermarker': False} are not expected by StableDiffusionPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 6/6 [00:20<00:00,  3.44s/steps]
/usr/local/lib/python3.10/dist-packages/spaces/zero/decorator.py:72: UserWarning: `enable_queue` parameter is now ignored and always set to `True`
  warnings.warn("`enable_queue` parameter is now ignored and always set to `True`")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f58c9b6b1b999e47a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
